In [1]:
import numpy as np
import open3d as o3d
import copy
from sklearn.neighbors import NearestNeighbors
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
num_tests = 100  
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
source = source.transform(trans_init)
my_target = np.asarray(target.points)
my_source = np.asarray(source.points[:my_target.shape[0]])

In [6]:
## Calculating the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions

def best_fit_transform(A, B):

    m = A.shape[1]
    A = A.T
    B = B.T
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)    
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # Getting the difference from mean
    A_mean = A - centroid_A
    B_mean = B - centroid_B

    Hom = A_mean @ np.transpose(B_mean)

    # find rotation
    U, S, Vt = np.linalg.svd(Hom)
    R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t.T

    return T, R, t

## Finding the nearest (Euclidean) neighbor in dst for each point in src

def nearest_neighbor(src, dst):
    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()

In [8]:
## The Iterative Closest Point method: finds best-fit transform that maps points A on to points B

def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):

    assert A.shape == B.shape
    m = A.shape[1]
    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1, A.shape[0]))
    dst = np.ones((m+1, B.shape[0]))
    src[:m, :] = np.copy(A.T)
    dst[:m, :] = np.copy(B.T)
    
   
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        
        distances, indices = nearest_neighbor(src[:m, :].T, dst[:m, :].T)
        T, _, _ = best_fit_transform(src[:m, :].T, dst[:m, indices].T)
        src = np.dot(T, src)  # update the current source
        mean_error = np.mean(distances) # check error
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error
    T, _, _ = best_fit_transform(A, src[:m, :].T)  # calculating final transformation

    return T, distances, i

def trans():

    A = my_source
    for i in range(num_tests):
        B = my_target        
        T, distances, iterations = icp(B, A, tolerance=0.1)
    print(T)
    return T
T = trans()

[[ 0.99999092 -0.00188073 -0.003824   -0.00243311]
 [ 0.00198401  0.99962844  0.02718542 -0.09554388]
 [ 0.00377145 -0.02719276  0.99962309  0.05347158]
 [ 0.          0.          0.          1.        ]]


In [5]:
def draw_registration_result(source, target, transformation):
    """
    param: source - source point cloud
    param: target - target point cloud
    param: transformation - 4 X 4 homogeneous transformation matrix
    """
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
    zoom=0.4459,
    front=[0.9288, -0.2951, -0.2242],
    lookat=[1.6784, 2.0612, 1.4451],
    up=[-0.3402, -0.9189, -0.1996])

draw_registration_result(target, source, T)